In [1]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3727  100  3727    0     0  49039      0 --:--:-- --:--:-- --:--:-- 49693
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200325 ...
Found existing installation: torch 1.4.0
Uninstalling torch-1.4.0:
Done updating TPU runtime: <Response [200]>
  Successfully uninstalled torch-1.4.0
Found existing installation: torchvision 0.5.0
Uninstalling torchvision-0.5.0:
  Successfully uninstalled torchvision-0.5.0
Copying gs://tpu-pytorch/wheels/torch-nightly+20200325-cp36-cp36m-linux_x86_64.whl...

Operation completed over 1 objects/83.4 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200325-cp36-cp36m-linux_x86_64.whl...

Operation completed over 1 objects/114.5 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-ni

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avishek-hotel/sentiments.csv
/kaggle/input/bert-base-uncased/vocab.txt
/kaggle/input/bert-base-uncased/config.json
/kaggle/input/bert-base-uncased/pytorch_model.bin
/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv
/kaggle/input/hotel-reviews/7282_1.csv
/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews.csv
/kaggle/input/bert-base-multilingual-uncased/vocab.txt
/kaggle/input/bert-base-multilingual-uncased/config.json
/kaggle/input/bert-base-multilingual-uncased/pytorch_model.bin


In [3]:
df=pd.read_csv("/kaggle/input/avishek-hotel/sentiments.csv",encoding='ISO-8859–1',header=0)
df.head()

,text,review
0,The premises are very peaceful and well mainta...,1
1,5 min drive out of Pisa or you need to catch t...,0
2,My family of 5 stayed at the residence San Ros...,1
3,We stayed here for a couple of nights on the w...,1
4,The accomodation was simple but more than adeq...,1


In [4]:
df1=pd.read_csv("/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews_Jun19.csv", usecols=['reviews.text', 'reviews.rating'])
df2=pd.read_csv("/kaggle/input/hotel-reviews/Datafiniti_Hotel_Reviews.csv", usecols=['reviews.text', 'reviews.rating'])
df3=pd.read_csv("/kaggle/input/hotel-reviews/7282_1.csv", usecols=['reviews.text', 'reviews.rating'])

In [5]:
train = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)

In [6]:
train["reviews.rating"]

0        3.0
1        4.0
2        3.0
3        5.0
4        2.0
        ... 
55907    5.0
55908    5.0
55909    5.0
55910    0.0
55911    0.0
Name: reviews.rating, Length: 55912, dtype: float64

In [7]:
train.loc[ train['reviews.rating'] <2.5, 'reviews.rating'] = -1

In [8]:
train.loc[ train['reviews.rating'] ==2.5, 'reviews.rating'] = 0


In [9]:
train.loc[ train['reviews.rating'] >2.5, 'reviews.rating'] = 1

In [10]:
 train=(train.dropna())

In [11]:
train["reviews.rating"]=train["reviews.rating"].astype(int)
train.head()

,reviews.rating,reviews.text
0,1,"This hotel was nice and quiet. Did not know, t..."
1,1,We stayed in the king suite with the separatio...
2,1,"Parking was horrible, somebody ran into my ren..."
3,1,Not cheap but excellent location. Price is som...
4,-1,If you get the room that they advertised on th...


In [12]:
train=train.rename(columns={"reviews.text": "text", "reviews.rating": "review"})
train.head()

,review,text
0,1,"This hotel was nice and quiet. Did not know, t..."
1,1,We stayed in the king suite with the separatio...
2,1,"Parking was horrible, somebody ran into my ren..."
3,1,Not cheap but excellent location. Price is som...
4,-1,If you get the room that they advertised on th...


In [13]:
t, valid, test = np.split(df.sample(frac=1), [int(.7*len(df)), int(.9*len(df))])

In [14]:
train=pd.concat([train,t], axis=0).reset_index(drop=True)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [15]:

valid=valid.rename(columns={"text": "comment_text", "review": "rating"})

In [16]:
train=train.rename(columns={"text": "comment_text", "review": "rating"})
train.head()

,rating,comment_text
0,1,"This hotel was nice and quiet. Did not know, t..."
1,1,We stayed in the king suite with the separatio...
2,1,"Parking was horrible, somebody ran into my ren..."
3,1,Not cheap but excellent location. Price is som...
4,-1,If you get the room that they advertised on th...


In [17]:
train=train[train["rating"]!=0]

In [18]:
train

,rating,comment_text
0,1,"This hotel was nice and quiet. Did not know, t..."
1,1,We stayed in the king suite with the separatio...
2,1,"Parking was horrible, somebody ran into my ren..."
3,1,Not cheap but excellent location. Price is som...
4,-1,If you get the room that they advertised on th...
...,...,...
96048,1,We stayed one night only is this quaint hotel ...
96049,1,WONDERFUL!!! IF YOU ARE LOOKING AT ANOTHER PLA...
96050,-1,I had stayed at this hotel for 3 nights in Dec...
96053,1,We have stayed in there for a night when we vi...


In [19]:
valid=valid[valid["rating"]!=0]

In [20]:
valid

,comment_text,rating
56648,We enjoyed Blue Sands very much. The staff wer...,1
38003,I love this hotel. As usual when staying in Sa...,1
6107,We??ve been in this hotel in February and was...,1
55541,Clean and nicely appointed. Rooms are a good s...,1
7731,Lovely hotel ideally situated for a short walk...,1
...,...,...
55765,Can't say enough about it. Perfect spot to see...,1
39663,This place is in need of some serious updating...,-1
32521,Stayed at Carleton Lodge for the first time th...,1
6650,Our room was looking the main street and it wa...,1


In [21]:
import os
import torch
import pandas as pd
from scipy import stats
import numpy as np

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib

import logging
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
import sys
from sklearn import metrics, model_selection

import warnings
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils

class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()
        
    #resets the values to zero
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    #updates the values of average
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [22]:
# a class to convert the dataset into lowercase
class BERTBaseUncased(nn.Module):
    #initialization function taking the path as a parameter
        def __init__(self, path):
            super(BERTBaseUncased, self).__init__()
            self.bert_path = path
            #Instantiate a pretrained pytorch model from a pre-trained model configuration.
            self.bert = transformers.BertModel.from_pretrained(self.bert_path)
            #randomly sets elements to zero to prevent overfitting.
            self.bert_drop = nn.Dropout(0.3) 
            #linear/outer layer as bert base model has 768*2 output features (bert base and multilingual) and 1 for binary classification
            self.out = nn.Linear(768 * 2, 1)
        
        def forward(self,ids,mask,token_type_ids):
            #o1 ,is the last hidden and we neglect pooler outputs of the bert 
            o1,_ = self.bert(ids,attention_mask=mask,token_type_ids=token_type_ids)
            
            apool = torch.mean(o1, 1)
            mpool, _ = torch.max(o1, 1)
            cat = torch.cat((apool, mpool), 1)

            bo = self.bert_drop(cat)
            output = self.out(bo)
            return output

In [23]:
#a class to accomodate the dataset to bert model
class BERTDatasetTrain:
    def __init__(self, comment_text,targets, tokenizer, max_length):
        self.comment_text = comment_text
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.targets=targets

    #getting the total length
    def __len__(self):
        return len(self.comment_text)
    
    #returns the token type ids from the dataset of the comment_text
    def __getitem__(self, item):
        #checking for the digits 
        comment_text = str(self.comment_text[item])
        #removing all the unwanted spaces
        comment_text = " ".join(comment_text.split())

        #encode 2 strings at a time hence 2nd string is none and add the CLS token
        inputs = self.tokenizer.encode_plus(comment_text,None,add_special_tokens=True,max_length=self.max_length,)
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        padding_length = self.max_length - len(ids)
        
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        
        return {'ids': torch.tensor(ids, dtype=torch.long),'mask': torch.tensor(mask, dtype=torch.long),'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),'targets': torch.tensor(self.targets[item], dtype=torch.float)}

In [24]:
mx = BERTBaseUncased(path="../input/bert-base-multilingual-uncased/")

In [25]:
def run():
    
    def loss_fn(outputs, targets):
        return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))

    def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
        model.train()
        for bi, d in enumerate(data_loader):
            ids = d["ids"]
            mask = d["mask"]
            token_type_ids = d["token_type_ids"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            optimizer.zero_grad()
            outputs = model(ids=ids,mask=mask,token_type_ids=token_type_ids)

            loss = loss_fn(outputs, targets)
            if bi % 10 == 0:
                xm.master_print(f'bi={bi}, loss={loss}')

            loss.backward()
            #Xla optimizer
            xm.optimizer_step(optimizer)
            if scheduler is not None:
                scheduler.step()

    def eval_loop_fn(data_loader, model, device):
        model.eval()
        fin_targets = []
        fin_outputs = []
        for bi, d in enumerate(data_loader):
            ids = d["ids"]
            mask = d["mask"]
            token_type_ids = d["token_type_ids"]
            targets = d["targets"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )

            targets_np = targets.cpu().detach().numpy().tolist()
            outputs_np = outputs.cpu().detach().numpy().tolist()
            fin_targets.extend(targets_np)
            fin_outputs.extend(outputs_np)    

        return fin_outputs, fin_targets

    
    MAX_LEN = 192
    TRAIN_BATCH_SIZE = 64
    EPOCHS = 15

    tokenizer = transformers.BertTokenizer.from_pretrained("../input/bert-base-multilingual-uncased/", do_lower_case=True)

    train_targets = train.rating.values
    valid_targets = valid.rating.values

    train_dataset = BERTDatasetTrain(comment_text=train.comment_text.values,targets=train_targets,tokenizer=tokenizer,max_length=MAX_LEN)

    train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset,num_replicas=xm.xrt_world_size(),rank=xm.get_ordinal(),shuffle=True)

    train_data_loader = torch.utils.data.DataLoader(train_dataset,batch_size=TRAIN_BATCH_SIZE,sampler=train_sampler,drop_last=True,num_workers=1)

    valid_dataset = BERTDatasetTrain(
        comment_text=valid.comment_text.values,
        targets=valid_targets,
        tokenizer=tokenizer,
        max_length=MAX_LEN
    )

    valid_sampler = torch.utils.data.distributed.DistributedSampler(
          valid_dataset,
          num_replicas=xm.xrt_world_size(),
          rank=xm.get_ordinal(),
          shuffle=False)

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=16,
        sampler=valid_sampler,
        drop_last=False,
        num_workers=1
    )

    device = xm.xla_device()
    model = mx.to(device)

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

    lr = 0.4 * 1e-5 * xm.xrt_world_size()
    num_train_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE / xm.xrt_world_size() * EPOCHS)
    xm.master_print(f'num_train_steps = {num_train_steps}, world_size={xm.xrt_world_size()}')

    #Adam Optimizer
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )

    for epoch in range(EPOCHS):
        para_loader = pl.ParallelLoader(train_data_loader, [device])
        train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler=scheduler)

        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)
        xm.save(model.state_dict(), "model.bin")
        
        # AUC tells how much model is capable of distinguishing between classes. 
        auc = metrics.roc_auc_score(np.array(t) >= 0.5, o)
        xm.master_print(f'AUC = {auc}')

In [26]:
# Start the training process
def _mp_fn(rank, flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    a = run()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

num_train_steps = 2660, world_size=8
bi=0, loss=0.5737982392311096


/pytorch/torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)
/pytorch/torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)
/pytorch/torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)
/pytorch/torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, Number alpha)
/pytorch/torch/csrc/utils/python_arg_parser.cpp:750: UserWarning: This overload of add_ is d

bi=10, loss=0.34277769923210144
bi=20, loss=0.28772637248039246
bi=30, loss=-0.5896278023719788
bi=40, loss=-0.04292493686079979
bi=50, loss=-0.565065324306488
bi=60, loss=-0.4308408200740814
bi=70, loss=-0.10912217944860458
bi=80, loss=-1.1541502475738525
bi=90, loss=-0.6131142377853394
bi=100, loss=-0.09173978865146637
bi=110, loss=-0.7218763828277588
bi=120, loss=-1.2719125747680664
bi=130, loss=-0.8977183103561401
bi=140, loss=-1.3622227907180786
bi=150, loss=0.4561169743537903
bi=160, loss=1.014007806777954
bi=170, loss=0.3053668737411499
AUC = 0.9876142131979696
bi=0, loss=-1.4210129976272583
bi=10, loss=0.1627686470746994
bi=20, loss=-0.8841155767440796
bi=30, loss=-2.7289726734161377
bi=40, loss=-1.6477148532867432
bi=50, loss=-1.5037308931350708
bi=60, loss=-1.2318180799484253
bi=70, loss=-0.6982350945472717
bi=80, loss=-3.109071731567383
bi=90, loss=-1.4408725500106812
bi=100, loss=-0.7926421165466309
bi=110, loss=-0.8305889964103699
bi=120, loss=-2.8438165187835693
bi=130, l